In [10]:
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt

In [11]:
DOMAINS = [
    "blocksworld",
    "ferry",
    "miconic",
    # "rovers",
    "satellite",
    # "transport",
]

In [12]:
baseline_data = pd.read_csv('baseline_results.csv')
baseline_data

,domain,problem,repeat,time,plan_length,plan_found,max_bound
0,blocksworld,0_01,0,1.584203,10.0,True,False
1,blocksworld,0_01,1,1.575279,10.0,True,False
2,blocksworld,0_01,2,1.613229,10.0,True,False
3,blocksworld,0_01,3,1.622800,10.0,True,False
4,blocksworld,0_01,4,1.605061,10.0,True,False
...,...,...,...,...,...,...,...
3595,satellite,2_30,5,NaN,NaN,False,False
3596,satellite,2_30,6,NaN,NaN,False,False
3597,satellite,2_30,7,NaN,NaN,False,False
3598,satellite,2_30,8,NaN,NaN,False,False


In [14]:
for domain in DOMAINS:
    print(domain)
    data = baseline_data[baseline_data['domain'] == domain]
    fig = px.scatter(data, x="problem", y="plan_length")
    fig.show()

blocksworld


ferry


miconic


satellite
